## API Requests and writing/reading to files

#### Reaching out to Weather History API

In [34]:
import requests
import pprint
import json
from dotenv import load_dotenv
import os

# Load environment variables, then use os module to retrieve it for usage in API requests
load_dotenv()

WEATHER_API = os.getenv("WEATHER_API")
history_url = "https://api.weatherapi.com/v1/history.json"


# List comprehension to account for dates to pass to API request parameters, cycle through with idx
dates = [f"2024-{n}-01" for n in range(2, 13)]
idx = 1

# Use to keep forecast data in environment by day
# history_data = []

# Begin looping through dates above, up to 2024-12-01
while idx < 11:

    # 'w' allows for you to write to a file and creates it if it doesn't exist (overrides, use 'a' for appending to end of file) 
    # '+' allows us to also read from files
    with open(f"history_api_responses/{dates[idx - 1]} to {dates[idx]}", "w+") as file:

        # Parameters to pass in to the request
        history_params = {
            "q": "91730",
            "dt": dates[idx - 1],
            "end_dt": dates[idx],
            "hour": 0,
            "key": WEATHER_API,
        }

        # Initialize variable to hold GET request response
        response_history = requests.get(history_url, params=history_params)

        # Write json to file 
        json.dump(response_history.json(), file, indent=4)

        # Start reading from top of file 
        file.seek(0)
        file_read = json.load(file)

        # Unpack previous data added to history_data (otherwise it'll be a nested list), 
        # and do the same for values within file_read this will continually append 
        # response data as we cycle through dates
        history_data = [*history_data, *file_read["forecast"]["forecastday"]]

        # Increment index of dates array
        idx += 1

# Use to ensure you're getting the response you expect in a 'prettified' form
pprint.pprint(history_data, sort_dicts=False)


[{'astro': {'moon_illumination': 66,
            'moon_phase': 'Waning Gibbous',
            'moonrise': '11:56 PM',
            'moonset': '10:16 AM',
            'sunrise': '06:48 AM',
            'sunset': '05:21 PM'},
  'date': '2024-02-01',
  'date_epoch': 1706745600,
  'day': {'avghumidity': 82,
          'avgtemp_c': 9.8,
          'avgtemp_f': 49.7,
          'avgvis_km': 9.1,
          'avgvis_miles': 5.0,
          'condition': {'code': 1240,
                        'icon': '//cdn.weatherapi.com/weather/64x64/day/353.png',
                        'text': 'Light rain shower'},
          'daily_chance_of_rain': 100,
          'daily_chance_of_snow': 0,
          'daily_will_it_rain': 1,
          'daily_will_it_snow': 0,
          'maxtemp_c': 12.2,
          'maxtemp_f': 54.0,
          'maxwind_kph': 13.7,
          'maxwind_mph': 8.5,
          'mintemp_c': 6.9,
          'mintemp_f': 44.4,
          'totalprecip_in': 0.83,
          'totalprecip_mm': 21.17,
          'total

#### Reaching out to Weather Forecast API

In [36]:
# Initialize URL & Parameters for request

forecast_url = "https://api.weatherapi.com/v1/forecast.json"

forecast_params = {
    "q": "91730",
    "days": "14",
    "hour": 0,
    "key": WEATHER_API,
}

# Open file with reading and writing capabilities
with open("forecast_api_responses/14_day_forecast.json", "w+") as file:

    # Capture API response
    response_forecast = requests.get(forecast_url, params=forecast_params)

    # Since this is a JSON response, keep formatting by using json.dump to write to file
    json.dump(response_forecast.json(), file, indent=4)

    # The writing above leaves file pointer where it left off, so setting .seek(0) allows
    # us to point back to the beginning of the file to read it in it's entirety
    file.seek(0)
    file_read = json.load(file)

    # Initialize an array to hold a specific key's values that we're going to use
    # Again, we use unpacking so we don't end up with a list nested needlessly
    forecast_data = [*file_read['forecast']['forecastday']]

# Check we have what we expect in our variable
pprint.pprint(forecast_data, sort_dicts=False)


[{'date': '2025-01-20',
  'date_epoch': 1737331200,
  'day': {'maxtemp_c': 16.2,
          'maxtemp_f': 61.2,
          'mintemp_c': 4.1,
          'mintemp_f': 39.4,
          'avgtemp_c': 10.0,
          'avgtemp_f': 50.0,
          'maxwind_mph': 21.3,
          'maxwind_kph': 34.2,
          'totalprecip_mm': 0.0,
          'totalprecip_in': 0.0,
          'totalsnow_cm': 0.0,
          'avgvis_km': 10.0,
          'avgvis_miles': 6.0,
          'avghumidity': 37,
          'daily_will_it_rain': 0,
          'daily_chance_of_rain': 0,
          'daily_will_it_snow': 0,
          'daily_chance_of_snow': 0,
          'condition': {'text': 'Sunny',
                        'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
                        'code': 1000},
          'uv': 0.7},
  'astro': {'sunrise': '06:54 AM',
            'sunset': '05:09 PM',
            'moonrise': '11:53 PM',
            'moonset': '10:31 AM',
            'moon_phase': 'Waning Gibbous',
            'mo

## Cleaning Data we received

### Using KNN Model

In [21]:
pd.options.display.width = 1000
pd.options.display.max_rows = None

df = pd.DataFrame()

for day in history_data:

    day_info = {
        "date": day["date"],
        "mintemp_f": day["day"]["mintemp_f"],
        "maxtemp_f": day["day"]["maxtemp_f"],
        "avgtemp_f": day["day"]["avgtemp_f"],
        "maxwind_mph": day["day"]["maxwind_mph"],
        "avgvis_miles": day["day"]["avgvis_miles"],
        "avghumidity": day["day"]["avghumidity"],
        "rained": day["day"]["daily_will_it_rain"],
    }

    df = df._append(day_info, ignore_index=True)

# df = pd.json_normalize(history_data)

pprint.pprint(df)


           date  mintemp_f  maxtemp_f  avgtemp_f  maxwind_mph  avgvis_miles  avghumidity  rained
0    2024-02-01       44.4       54.0       49.7          8.5           5.0           82       1
1    2024-02-02       42.1       52.9       47.9          9.8           6.0           79       1
2    2024-02-03       42.8       53.3       47.9          8.9           5.0           71       1
3    2024-02-04       47.9       55.5       51.1         12.5           4.0           73       1
4    2024-02-05       48.6       51.3       50.1          7.8           4.0           95       1
5    2024-02-06       46.7       50.7       48.4          7.4           5.0           93       1
6    2024-02-07       44.1       49.5       46.6          9.8           5.0           84       1
7    2024-02-08       39.7       49.4       44.8         13.4           6.0           73       1
8    2024-02-09       41.4       51.3       46.3          7.4           6.0           72       1
9    2024-02-10       38.7    

In [ ]:
# import pandas as pd
# import sklearn

# df_history = pd.js
